In [60]:
# %load init.py
#import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
import dill
#df = pd.read_csv('../data/complaints.csv')
#dill.dump_session('read_data.db')  # so that I don't have to parse the csv every time
dill.load_session('read_data.db')

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1517620 entries, 0 to 1517619
Data columns (total 18 columns):
Date received                   1517620 non-null object
Product                         1517620 non-null object
Sub-product                     1282455 non-null object
Issue                           1517620 non-null object
Sub-issue                       957685 non-null object
Consumer complaint narrative    499118 non-null object
Company public response         572138 non-null object
Company                         1517620 non-null object
State                           1492059 non-null object
ZIP code                        1374900 non-null object
Tags                            206318 non-null object
Consumer consent provided?      898083 non-null object
Submitted via                   1517620 non-null object
Date sent to company            1517620 non-null object
Company response to consumer    1517619 non-null object
Timely response?                1517620 non-null obj

In [62]:
dfreduced = pd.DataFrame()
dfreduced['Company'] = df['Company']
dfreduced['Responsive'] = df['Timely response?']
dfreduced

,Company,Responsive
0,TRANSWORLD SYSTEMS INC,Yes
1,Experian Information Solutions Inc.,Yes
2,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Yes
3,"Diversified Consultants, Inc.",Yes
4,HYUNDAI CAPITAL AMERICA,Yes
...,...,...
1517615,NISSAN MOTOR ACCEPTANCE CORPORATION,Yes
1517616,"CMRE Financial Services, Inc.",Yes
1517617,Finance of America Mortgage LLC,Yes
1517618,JPMORGAN CHASE & CO.,Yes


In [64]:
nTop = 10
dfg = dfreduced.groupby(['Company']).count()[['Responsive']].sort_values('Responsive', ascending=False)
dfg = dfg.reset_index()
dfgTop = dfg.head(nTop)
dfgTop

,Company,Responsive
0,"EQUIFAX, INC.",152828
1,Experian Information Solutions Inc.,135725
2,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",132287
3,"BANK OF AMERICA, NATIONAL ASSOCIATION",88680
4,WELLS FARGO & COMPANY,76625
5,JPMORGAN CHASE & CO.,66665
6,"CITIBANK, N.A.",54764
7,CAPITAL ONE FINANCIAL CORPORATION,41312
8,"Navient Solutions, LLC.",31845
9,Ocwen Financial Corporation,29265


In [65]:
dfreducedTop = dfreduced.loc[dfreduced['Company'].isin(dfgTop['Company'])]
dfreducedTop

,Company,Responsive
1,Experian Information Solutions Inc.,Yes
2,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Yes
7,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Yes
11,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Yes
12,Experian Information Solutions Inc.,Yes
...,...,...
1517608,"CITIBANK, N.A.",Yes
1517612,"CITIBANK, N.A.",Yes
1517614,"Navient Solutions, LLC.",Yes
1517618,JPMORGAN CHASE & CO.,Yes


In [69]:
dfreducedOther = dfreduced.loc[dfreduced['Company'].isin(dfgTop['Company']) == False]
dfreducedOther = dfreducedOther.assign(Company = 'Other')
dfreducedOther

,Company,Responsive
0,Other,Yes
3,Other,Yes
4,Other,Yes
5,Other,Yes
6,Other,Yes
...,...,...
1517611,Other,Yes
1517613,Other,Yes
1517615,Other,Yes
1517616,Other,Yes
